In [ ]:
!pip install git+https://github.com/boudinfl/pke.git
!pip install matplotlib
!python -m spacy download en_core_web_sm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import exists

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/PaperA3/Corpus/corpus_BOC_NegCUI.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df.dropna()
        

In [5]:
dd=pd.read_csv('/content/drive/MyDrive/PaperA3/Diagnoses_labels/Dictionary.csv')
dictionary = dict(zip(dd.first_cuid, dd.canonical_name))

In [6]:
import ast
def complex_function( x):
    res =x.split(",")
    tms_text=''    
    for i in range(len(res)):
        try:  
            if(res[i].upper().startswith("NOT")):
              tms_text=tms_text +'NOT '+dictionary[res[i][4:].upper()] +', '
            else:
              tms_text=tms_text +dictionary[res[i].upper()] +', '  
        except :
              print(res[i])    
    return tms_text 

In [7]:
df=df.dropna()
df['Text'] = df.apply(lambda x: complex_function(x['term']), axis=1)
df.shape

(26357, 3)

In [8]:
filename="/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_1.csv"
df_remain=pd.read_csv(filename)
df_remain=df_remain[['label','Text']]
for i in range(2,30):
  filename="/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_"+str(i)+".csv"
  
  if (exists(filename)==True):
          
          df2=pd.read_csv(filename)
          df2=df2[['label','Text']]
          
          df_remain=pd.concat([df_remain, df2])
          print(filename,len(df2),len(df_remain))
ls_remain=df_remain.label.astype(float).astype(int).unique().tolist() 

/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_2.csv 1000 1001
/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_3.csv 1000 2001
/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_4.csv 1000 3001
/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_5.csv 1000 4001
/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_6.csv 1000 5001
/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_7.csv 1000 6001
/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_8.csv 1000 7001
/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_9.csv 1000 8001
/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_10.csv 1000 9001
/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_11.csv 1000 10001
/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_12.csv 1000 11001
/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_13.csv 1000 12001
/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_14.csv 1000 13001
/co

In [9]:
df_remain.head()

,label,Text
0,157267.0,"congestive heart failure, refractory anemias, ..."
0,123103.0,"nicotinamide adenine dinucleotide, refractory ..."
1,139931.0,"right renal artery, left renal artery, renal m..."
2,198295.0,"congestive heart failure, lung diseases, hyper..."
3,177962.0,"left ventricular function, left ventricular hy..."


In [10]:
df1=df[['label','Text']]
df1.label=df1.label.astype(float).astype(int)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [11]:


df3=df1[~df1.label.isin(ls_remain)]
df3.shape

(1028, 2)

In [12]:
import pke

# initialize a TopicRank keyphrase extraction model
extractor = pke.unsupervised.PositionRank()

### step-2: what we need now is a sample document

In [13]:
# sample document (2040.abstr from the Inspec dataset)
data_all_new= []
for j in range(len(df3)):
    print(len(df3)-j)
    sample = df3.iloc[j].Text.replace("\n", " ")
    extractor.load_document(input=sample, language='en')
    extractor.candidate_selection()
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=20, stemming=False)
    keyphrases
    # for each of the best candidates
    ls=[]
    for i, (candidate, score) in enumerate(keyphrases):
        ls.append(candidate)
        # print out the its rank, phrase and score
      #print("rank {}: {} ({})".format(i, candidate, score))
    str_tmp = ', '.join([str(elem) for elem in ls])
    data_all_new.append( [df.iloc[j].label,str_tmp])
    if (j % 1000 ==0):
          filename=int(j/1000+31)
          pd.DataFrame(data_all_new,columns=['label','Text']).to_csv('/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_'+str(filename)+'.csv')
          data_all_new= []

pd.DataFrame(data_all_new,columns=['label','Text']).to_csv('/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_'+str(filename)+'.csv')

1028
1027
1026
1025
1024
1023
1022
1021
1020
1019
1018
1017
1016
1015
1014
1013
1012
1011
1010
1009
1008
1007
1006
1005
1004
1003
1002
1001
1000
999
998
997
996
995
994
993
992
991
990
989
988
987
986
985
984
983
982
981
980
979
978
977
976
975
974
973
972
971
970
969
968
967
966
965
964
963
962
961
960
959
958
957
956
955
954
953
952
951
950
949
948
947
946
945
944
943
942
941
940
939
938
937
936
935
934
933
932
931
930
929
928
927
926
925
924
923
922
921
920
919
918
917
916
915
914
913
912
911
910
909
908
907
906
905
904
903
902
901
900
899
898
897
896
895
894
893
892
891
890
889
888
887
886
885
884
883
882
881
880
879
878
877
876
875
874
873
872
871
870
869
868
867
866
865
864
863
862
861
860
859
858
857
856
855
854
853
852
851
850
849
848
847
846
845
844
843
842
841
840
839
838
837
836
835
834
833
832
831
830
829
828
827
826
825
824
823
822
821
820
819
818
817
816
815
814
813
812
811
810
809
808
807
806
805
804
803
802
801
800
799
798
797
796
795
794
793
792
791
790
789
788
787
786

In [10]:
pd.DataFrame(data_all_new,columns=['label','Text']).to_csv('/content/drive/MyDrive/PaperA3/PositionRank/PositionRank_20_'+'_'+str(28)+'.csv')
